## ACTD 2025-10

### Clases 13-14: redes neuronales para clasificación

- Redes densas
- Redes para clasificación
- Procesamiento de datos continuos y categóricos
- Funciones de activación y pérdida

Construyamos un modelo para clasificar flores con el dataet iris usando Keras y Tensorflow

In [1]:
import tensorflow as tf

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Importemos y exploremos los datos

In [2]:
iris = load_iris()

print(iris.data[0:3,])
print(iris.target[0:3,])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
[0 0 0]


Codifiquemos la variable de salida como categórica con 3 clases

In [3]:
y = tf.keras.utils.to_categorical(iris.target, num_classes=3)

print(y[0:3,])
print(y.shape)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
(150, 3)


Dividamos los datos en entrenamiendo, validación y prueba usando Scikit-Learn

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    iris.data, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42)

X_train.shape

Reescalemos los datos (inputs X) usando una normalización estándar

In [ ]:
std_scl = StandardScaler()
std_scl.fit(X_train)

print(X_train[0:3,])
X_train = std_scl.transform(X_train)
print(X_train[0:3,])
X_valid = std_scl.transform(X_valid)
X_test = std_scl.transform(X_test)

In [ ]:
X_train.dtype

Construyamos el modelo con una capa de entrada, una capa densa con 5 neuronas y una capa de salida con 3 neuronas (1 por cada categoría de la variable de respuesta)

In [ ]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(4,)))
model.add(tf.keras.layers.Dense(5, activation="relu"))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

In [ ]:
model.summary()

Exploremos las capas del modelo, sus pesos y sesgos

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
weights, biases = hidden1.get_weights()
weights

In [ ]:
weights.shape

In [ ]:
import pandas as pd
pd.DataFrame(weights)

In [ ]:
biases

In [ ]:
biases.shape

In [ ]:
pd.DataFrame(biases)

Compilemos el modelo usando entropía categórica cruzada como función de pérdida

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Entrenemos el modelo

In [ ]:
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid))

## Datos MNIST fashion
Ahora entrenemos otro modelo de clasificación, esta vez para datos MNIST fashion

In [ ]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train.shape

In [ ]:
pd.DataFrame(X_train[0,:])

Reescalemos los datos para que cada pixel tenga un valor entre 0 y 1

In [ ]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [ ]:
pd.DataFrame(X_train[0,:])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

In [ ]:
y_train[0]

Definamos los nombres de las 10 categorías

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

class_names[y_train[0]]

Grafiquemos algunos ejemplos de las imágenes en el dataset

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]])
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

Definimos el modelo con
- 1 capa de entrada
- 1 operación de adelgazamiento (flatten)
- 2 capas densas con función de activación ReLU
- 1 capa de salida con función de activación softmax (clasificación) con tantas neuronas como categorías tenemos etiquetadas (10)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
weights, biases = hidden1.get_weights()
weights.shape

In [ ]:
biases.shape

Compilamos el modelo

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

Entrenamos el modelo

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Probando el modelo

In [ ]:
test_case = 20
plt.imshow(X_test[test_case], cmap="binary")
plt.axis('off')
plt.show()
print(f'Actual label: {class_names[y_test[test_case]]}')
dftest = pd.DataFrame(model.predict(X_test[[test_case]]), columns=class_names)
dftest